In [1]:
!pip install -q langchain openai faiss-cpu pypdf pymupdf tiktoken
!pip install -q langchain langchain-core langchain-text-splitters
!pip install -q fastembed faiss-cpu langchain-community
!pip install -q langchain-groq
!pip install fitz
!pip install -q pypdf




[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached httplib2-0.31.0-py3-none-any.whl.metadata (2.2 kB)
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB 1.6 MB/s eta 0:00:00
  Using cached pyparsing-3.3.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached lxml-6.0.2-cp311-cp311-win_amd64.whl.metadata (3.7 kB)
Using cached httplib2-0.31.0-py3-none-any.whl (91 kB)
   ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/3.3 MB 9.8 MB/s eta 0:00:01
   ---------------- ----------------------- 1.3/3.3 MB 14.2 MB/s eta 0:00:01
   ----------------- ---------------------- 1.4/3.3 MB 11.4 MB/s eta 0:00:01
   ------------------------------ --------- 2.5/3.3 MB 13.5 MB/s eta 0:00:01
   -------------------------------------- - 3.1/3.3 MB 13.4 MB/s eta 0:00:01
   ---------------------------------------- 3.3/3.3 MB 12.3 MB/


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from pypdf import PdfReader

def extract_text(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text


In [11]:
gita_text  = extract_text("data/gita.pdf")
bible_text = extract_text("data/bible.pdf")
quran_text = extract_text("data/quran.pdf")

In [12]:
def normalize(text, source):
    lines = text.split("\n")
    cleaned = []
    for i, line in enumerate(lines):
        line = line.strip()
        if line:
            cleaned.append(f"[{source.upper()}:{i}] {line}")
    return "\n".join(cleaned)


In [13]:
gita_text  = normalize(gita_text,  "gita")
bible_text = normalize(bible_text, "bible")
quran_text = normalize(quran_text, "quran")


In [15]:
import os

os.makedirs("data/gita", exist_ok=True)
os.makedirs("data/bible", exist_ok=True)
os.makedirs("data/quran", exist_ok=True)

with open("data/gita/gita.txt", "w", encoding="utf-8") as f:
    f.write(gita_text)
with open("data/bible/bible.txt", "w", encoding="utf-8") as f:
    f.write(bible_text)
with open("data/quran/quran.txt", "w", encoding="utf-8") as f:
    f.write(quran_text)

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document


splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100
)

docs = []

for source in ["gita", "bible", "quran"]:
    text = open(f"data/{source}/{source}.txt", encoding="utf-8").read()
    chunks = splitter.split_text(text)

    for chunk in chunks:
        docs.append(
            Document(
                page_content=chunk,
                metadata={"source": source}
            )
        )

len(docs)

r:\Projects\gita_gpt\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


13693

In [17]:
!pip install -q python-dotenv



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
from dotenv import load_dotenv
import os

load_dotenv()

assert os.getenv("HF_TOKEN") is not None, "HF_TOKEN not loaded"


In [20]:
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = FastEmbedEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
)

db = FAISS.from_documents(docs, embeddings)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]r:\Projects\gita_gpt\venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\asus\AppData\Local\Temp\fastembed_cache\models--qdrant--all-MiniLM-L6-v2-onnx. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 5 files: 100%|██████████| 5/5 [00:08<00

In [21]:
db.save_local("scripture_faiss")
#db = FAISS.load_local("scripture_faiss", embeddings) -> import the embeddings created

In [22]:
SYSTEM_PROMPT = """
You are a comparative scripture assistant.

Rules:
1. Use ONLY the retrieved text.
2. Never merge teachings.
3. Answer separately for each scripture.
4. If a scripture does not address the question, say:
   "No direct reference found."

Format EXACTLY like this:

Gita says:
- ...

Bible says:
- ...

Quran says:
- ...
"""



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
import os
ChatGroq = os.environ["GROQ_API_KEY"]


In [33]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",   # free, strong enough
    temperature=0
)

retriever = db.as_retriever(search_kwargs={"k": 15})


In [34]:
def ask(question):
    # LangChain retrievers are "Runnable" in newer versions; use .invoke()
    if hasattr(retriever, "get_relevant_documents"):
        docs = retriever.get_relevant_documents(question)
    else:
        docs = retriever.invoke(question)

    context = "\n\n".join(
        f"[{d.metadata['source'].upper()}]\n{d.page_content}"
        for d in docs
    )

    prompt = f"""{SYSTEM_PROMPT}

Context:
{context}

Question:
{question}
"""

    response = llm.invoke(prompt)
    return response.content


In [ ]:
print(ask("What do the scriptures say about forgiveness?"))


Gita says:
- No direct reference found.

Bible says:
- He forgives all your iniquity; he heals all your diseases. (BIBLE:56544)
- And forgive us our debts, as we also have forgiven our debtors. (BIBLE:90015)
- May you hear in heaven and forgive the sin of your people Israel. (BIBLE:40708)
- If your people Israel are defeated before an enemy, because they have sinned against you, and they return to you and praise your name, and they pray and plead for mercy before you in this temple, may you hear in heaven and forgive. (BIBLE:40703-40705)

Quran says:
- Whoso does right, does it for his own soul; and whoso does wrong, does so to its detriment. Then to your Lord will you all be brought back. (QURAN:30813-30817)
- Say to those who believe, that they exercise forgiveness towards those who do not expect the promised days of Allah to come to pass, that He may requite a people for what they earn. (QURAN:30819-30821)
- And verily, We gave the children of Israel the Book, and sovereignty, (QURA

: 